In [1]:
from Plot_AQI import avg_data_2013,avg_data_2014,avg_data_2015,avg_data_2016,avg_data_2017,avg_data_2018
#from my_Plot_AQI import avg_data ModuleNotFoundError: No module named 'my_Plot_AQI'
import requests
import sys
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import csv

In [2]:
def met_data(month, year):
    
    file_html = open('Data/Html_Data/{}/{}.html'.format(year,month), 'rb')
    plain_text = file_html.read()

    tempD = []
    finalD = []

    soup = BeautifulSoup(plain_text, "lxml")
    for table in soup.findAll('table', {'class': 'medias mensuales numspan'}):
        for tbody in table:
            for tr in tbody:
                a = tr.get_text()
                tempD.append(a)#had all the table

    rows = len(tempD) / 15

    for times in range(round(rows)):
        newtempD = []
        for i in range(15):
            newtempD.append(tempD[0])#picking each cell in a row
            tempD.pop(0)
        finalD.append(newtempD)#appending the full row

    length = len(finalD)

    finalD.pop(length - 1)#last row ie 33 ['Monthly means and totals:',
    finalD.pop(0)# 1st Row

    for a in range(len(finalD)):
        finalD[a].pop(6)
        finalD[a].pop(13)
        finalD[a].pop(12)
        finalD[a].pop(11)
        finalD[a].pop(10)
        finalD[a].pop(9)
        finalD[a].pop(0)

    return finalD#final DataFrame 7columns 365/364 rows


In [17]:
def data_combine(year, cs):
    for a in pd.read_csv("Data/Real-Data/real_" + str(year) + ".csv" , chunksize=cs):
        df = pd.DataFrame(data=a)
        mylist = df.values.tolist()
    return mylist



In [4]:
if __name__ == "__main__":
    if not os.path.exists("Data/Real-Data"):
        os.makedirs("Data/Real-Data")
    for year in range(2013,2019):
        final_data = []
        with open('Data/Real-Data/real_' + str(year) +'.csv' ,'w') as csvfile:
            wr = csv.writer(csvfile, dialect='excel')
            wr.writerow(
                    ['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM' ,'PM 2.5']
                    )
        for month in range(1, 13):
            temp = met_data(month,year)
            final_data = final_data + temp
        
        pm = getattr(sys.modules[__name__], 'avg_data_{}'.format(year))()
        
        if len(pm) == 364:
            pm.insert(364,'--')
        
        for i in range(len(final_data) - 1):
            final_data[i].insert(8, pm[i]) #in 8th column insert the pm 
            
        with open('Data/Real-Data/real_' + str(year) + '.csv', 'a') as csvfile:
            wr = csv.writer(csvfile, dialect='excel')
            for row in final_data:
                flag = 0
                for elem in row:
                    if elem == '' or elem == '-':
                        flag = 1
                if flag != 1:
                    wr.writerow(row)      

In [19]:
data_2013 = data_combine(2013,600)
data_2014 = data_combine(2014,600)
data_2015 = data_combine(2015,600)
data_2016 = data_combine(2016,600)
data_2017 = data_combine(2017,600)
data_2018 = data_combine(2018,600)

total = data_2013 + data_2014 + data_2015 + data_2016 + data_2017 + data_2018
vtotal = data_2013 + data_2014 + data_2015 + data_2016

with open('Data/Real-Data/Real_Combine.csv' , 'w') as csvfile:
    wr = csv.writer(csvfile , dialect='excel')
    wr.writerow(
            ['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM 2.5'])
    wr.writerows(total)
        
        
with open('Data/Real-Data/VReal_Combine.csv' , 'w') as csvfile:
    wr = csv.writer(csvfile , dialect='excel')
    wr.writerow(
            ['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM 2.5'])
    wr.writerows(vtotal)


In [23]:
df = pd.read_csv('Data/Real-Data/Real_Combine.csv')
print(df.shape)
df.head()

(1090, 9)


,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


# Rough Code

In [16]:
file_html = open('Data/Html_Data/2017/1.html')
plain_text = file_html.read()

tempD = []
finalD = []
soup = BeautifulSoup(plain_text,"lxml")
for table in soup.findAll('table',{'class':'medias mensuales numspan'}):
    for tbody in table:
        for tr in tbody:
            a = tr.get_text()
            tempD.append(a)#had all table
rows = len(tempD)/15

for times in range(round(rows)):
    newtempD = []
    for i in range(15):
        newtempD.append(tempD[0])#picking each cell in a row
        tempD.pop(0)
    finalD.append(newtempD)#appending the whole row
    
length = len(finalD)

finalD.pop(length - 1)#last row ie 33 ['Monthly means and totals:',
finalD.pop(0) #1st row

for a in range(len(finalD)):
    finalD[a].pop(6)
    finalD[a].pop(13)
    finalD[a].pop(12)
    finalD[a].pop(11)
    finalD[a].pop(10)
    finalD[a].pop(9)
    finalD[a].pop(0)


In [17]:
finalD

[['21', '27.6', '14.6', '-', '61', '6', '2.2', '3.5'],
 ['21.6', '27.7', '17', '-', '59', '6.8', '2', '3.5'],
 ['22.3', '29', '16.1', '-', '48', '6.9', '2.8', '3.5'],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['20.9', '29', '16', '-', '66', '5.5', '2.2', '3.5'],
 ['20.9', '27.8', '16', '-', '62', '6.3', '3', '5.4'],
 ['21', '28.5', '15', '-', '58', '6.8', '3', '5.4'],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['21.2', '28', '16.9', '-', '66', '5.5', '2.2', '3.5'],
 ['20.7', '28.3', '17.2', '-', '61', '6.3', '2.2', '3.5'],
 ['20.7', '27.2', '15.5', '-', '61', '6.3', '3.3', '5.4'],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['', '', '', '', '', '', '', ''],
 ['20.4', '28.2', '15.8', '-', '64', '6.3', '2.8', '7.6'],
 ['21.7', '27.6', '15.3', '-', '64', '6.3', '3.5', '5.4'],
 ['21.7', '28.9', '18.5', '-', '68', '6.3', '5.2', '11.1'],
 [

In [13]:
pm = getattr(sys.modules[__name__], 'avg_data_2013')()

In [15]:
len(pm)

365

In [12]:
year = 2013
for a in pd.read_csv('Data/Real-Data/real_' + str(year) + '.csv' , chunksize=600):
    df = pd.DataFrame(data=a)
    mylist = df.values.tolist()
    print(len(mylist))
    

185


In [13]:
df = pd.read_csv('Data/Real-Data/real_2013.csv')
print(type(df))
print(len(df.values.tolist()))

<class 'pandas.core.frame.DataFrame'>
185
